# 2023年 大規模言語モデル 第5回演習  テーマ：LLMのFine-Tuning

演習目的：LLMで追加学習、評価を行えるようになること

## 目次
1. 準備
1. Fine-Tuningするモデルについて
1. データ準備
1. 学習方法
1. 評価指標

## 1. 準備

学習の推移を記録するためにwandbをimport

In [ ]:
#学習状況を可視化
import wandb
import os
#set it manually with the WANDB_NOTEBOOK_NAME environment variable
os.environ["WANDB_NOTEBOOK_NAME"] = "stablelm_qlora.ipynb"
wandb.login()
wandb.init(project="test-stablelm-7b")

wandb: Currently logged in as: hikaru-n (pjt-s). Use `wandb login --relogin` to force relogin


推論などの時にwarningが出ないようにするためにloggingを設定

In [ ]:
from transformers import logging
logging.set_verbosity(logging.CRITICAL)

## 2. Fine-Tuningするモデルについて
Stability AI Japanから商業利用可能なJapanese StableLM Alphaの提供が開始しました(2023/8/10).  
- Japanese StableLM Base Alpha 7B 商用利用可能
- Japanese StableLM Instruct Alpha 商業利用不可

今回は，Japanese StableLM Base Alpha 7Bを利用し商業利用可能なモデルをFine-Tuningします．

まずは，Japanese StableLM Base Alpha 7Bをロードして推論してみましょう  
モデル名を指定します

In [1]:
# The model from the Hugging Face hub
model_name = "4bit/japanese-stablelm-base-alpha-7b-s"

モデルのパラメータを設定する

In [2]:
import torch
from transformers import BitsAndBytesConfig
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
# Load the entire model on the GPU 0
device_map = {"": 0}

# set up the model parameters
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print("=" * 80)
    print("Your GPU supports bfloat16: accelerate training with bf16=True")
    print("=" * 80)

bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)

ModuleNotFoundError: ignored

モデルをロードする

In [ ]:
# Load base model
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizerをロードする

In [ ]:
# Load LLaMA tokenizer　for japanese stableLM
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1", additional_special_tokens=['▁▁'])
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

モデルに入力するテキストを設定する

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
# Run text generation pipeline with our next model
text = "西郷隆盛はどんな人物ですか？"

prompt = "{bos_token}{b_inst} {prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    prompt=text,
    e_inst=E_INST,
)
print(prompt)


pipeline関数を使いモデルの出力を得る

In [ ]:
from transformers import pipeline
pipe = pipeline(task="text-generation",
                 model=model,
                 tokenizer=tokenizer,
                 max_new_tokens=30
                )
result = pipe(prompt)

出力内容を確認

In [ ]:
print(result[0]["generated_text"])
#[/INST]以降をprintする

print('\n出力だけを表示')
print('-'*80)
print(result[0]["generated_text"].split(E_INST)[1])
print('-'*80)

今回の演習ではこのJapanese StableLM Base Alpha 7BをFine-Tuningし,
1. 自然な日本語を生成させる
1. JGLUEのベンチマークで高いスコアを出す

これら２つを目標にFine Tuningとします.   
以下のllama2の論文からの画像で学習の流れを示しています.  
今回の演習では右下の菱形のSupervised Fine Tuningの部分を実際に動かす流れを確認します.  
PretrainingとRLHFの中間の部分です.

<p align="center">
  <img src="./figure_and_table/figure4_llama2.png" alt="404" width="600"/>
</p>

[1]の論文から引用

まずは1.自然な日本語を生成させるためにfine tuningの流れを行います.  
その後同様の流れで2.JGLUEでのfine tunigを行います.

流れとしては "データの準備" -> "学習" -> "評価" です.  

## 3. データ準備

### 日本語データはどうやって選ぶ?
選ぶポイント  
a. データの質  
b. 目的に沿った多様なデータ  
c. ライセンス  

### a.データの質
自然な日本語かどうか  
翻訳データセットが多いため翻訳の精度の低いと思われるデータセットには注意が必要
- kunishou/hh-rlhf-49k-jaから抜粋  
    "ケチャップの汚れはデニムのズボンから出てきますか？"  
    " Do ketchup stains come out of denim pants? "

### b.目的に沿った多様なデータ
目的に沿ったデータを選択.今回は日本語でのQandAが学習できるデータを選択  
学習させるデータが特殊なタスクに特化しないようなデータを選択することが重要

### c.ライセンス
モデルを商業利用可能な状態に保つためには、商業利用可能なデータでの学習が必要

### 実際のデータを収集
以下のデータは今回使えそうな日本語質問回答のデータセット
- kunishou/databricks-dolly-15k-ja (15k)
- kunishou/oasst1-89k-ja (55k)
- izumi-lab/llm-japanese-dataset (9.05M)
- shumpei2525/fine_tuning521k-ja (521k)
- sudy-super/CoTangent (100)
- livedoor ニュースコーパス

以下にhuggingface Datasetの[kunishou/databricks-dolly-15k-ja](https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja)というデータの画面を示す.  
画面から目的、ライセンス、データの質を確認.


<p align="center">
  <img src="./figure_and_table/HF_databricks-dolly-15k-ja.png" alt="404" width="800"/>
</p>

[hugging face](https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja)のデータセットの画面

このデータセットをLoad

In [ ]:
from datasets import load_dataset
dataset = load_dataset("kunishou/databricks-dolly-15k-ja", split="train")

Found cached dataset json (/home/nagazumi/.cache/huggingface/datasets/kunishou___json/kunishou--databricks-dolly-15k-ja-6459e484ab1ecf5e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [ ]:
#datasetの中身を確認
dataset

Dataset({
    features: ['input', 'instruction', 'output', 'index', 'category'],
    num_rows: 15015
})

In [ ]:
dataset[2]

{'input': '',
 'instruction': 'ラクダはなぜ水なしで長く生きられるのか？',
 'output': 'ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。',
 'index': '2',
 'category': 'open_qa'}

演習の時間に制限があるためdatasetを小さくする

In [ ]:
# datasetの前1000個抽出
dataset = dataset.select(range(1000))

datasetを学習時に使える入力文の形に変換します.

In [ ]:
# プロンプトテンプレートの準備
def generate_prompt(data_point):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。質問に答えてください。"

    if data_point["input"]:
        text =  f"""{data_point["input"]}\n{data_point["instruction"]}"""
    else:
        text =  f"""{data_point["instruction"]}"""

    prompt = "{bos_token} {b_inst} {prompt} {e_inst} {output}".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
        output=data_point["output"],
    )
    return prompt

In [ ]:
dataset[2]

{'input': '',
 'instruction': 'ラクダはなぜ水なしで長く生きられるのか？',
 'output': 'ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。',
 'index': '2',
 'category': 'open_qa'}

datasetとして入らない要素を削除する関数を定義します

In [ ]:
# テキスト列の追加
def add_text(example):
    example["text"] = generate_prompt(example)
    del example["index"]
    del example["category"]
    del example["instruction"]
    del example["input"]
    del example["output"]
    return example

In [ ]:
dataset = dataset.map(add_text)

Loading cached processed dataset at /home/nagazumi/.cache/huggingface/datasets/kunishou___json/kunishou--databricks-dolly-15k-ja-6459e484ab1ecf5e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-4cd2ba916a27d44c.arrow


Datasetがpromptの形式に変換できているか確認

In [ ]:
print('-'*180)
print(dataset[2])
print('-'*180)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'text': '<|startoftext|> [INST] ラクダはなぜ水なしで長く生きられるのか？ [/INST] ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。'}
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


## 4.学習方法

### 学習手法の説明

今回の演習ではPEFTの手法の中でAdapter型の手法LoRA, QLoRAの実装を紹介する.  

### LoRA
LoRA(Low-Rank Adaptation)は以下のようにTransformerの中間層に行列を追加し  
追加した層のパラメタのみを学習する.   

<p align="center">
  <img src="./figure_and_table/AACL_2022_tutorial_PLMs_LoRA.png" alt="404" width="600"/>
</p>

この追加した箇所は以下の図のように並列に追加される.


<p align="center">
  <img src="./figure_and_table/figure1_lora.png" alt="404" width="200"/>
</p>

実装は以下のようになる.  

```python
input_dim = 768  # 入力次元d
output_dim = 768  # 出力次元d

rank = 8  # The rank 'r' for the low-rank adaptation パラメタの次元

W = ... # 事前学習で学習したFeed-Forwardのパラメタ　次元は input_dim x output_dim

W_A = nn.Parameter(torch.empty(input_dim, rank)) # LoRA weight A 次元は input_dim x rank
W_B = nn.Parameter(torch.empty(rank, output_dim)) # LoRA weight B 次元は rank x output_dim

alpha = 1.0 # 追加した層の結果を調整するハイパーパラメタ
...

# 通常のFeed-Forwardの実装
def regular_forward_matmul(x, W):
    h = x @ W
return h

# LoRAの実装
def lora_forward_matmul(x, W, W_A, W_B):
    h = x @ W  # regular matrix multiplication
    h += x @ (W_A @ W_B)*alpha # alphaで追加した層の出力を調整するハイパーパラメタ
return h
```


メリット
- 学習の速度向上：学習時更新するパラメタが減るため学習時間が早くなる.(推論は高速化しない)  
- メモリ使用量の削減：学習時更新するパラメタが減る.  
上記のプログラムを例にすると学習するパラメタのオーダーは W_A,W_B の合計で$2*d*rank$となる.  
そのため、保持しておくメモリ使用量が減る.(推論は小メモリ化しない)  

使用するメモリの概算  
- モデルサイズ：7b
- パラメタサイズ：float16(16ビット浮動小数点数 詳しくは[torch.flaot16](https://note.nkmk.me/python-pytorch-dtype-to/))  

総メモリ:7b * 16bit / 8  = 14GB  


14GBのモデルサイズで学習するとバッチサイズ4とすると14GB*4 = 56GBのメモリが必要.  

LoRA実装リンク:[peft](https://github.com/huggingface/peft)  


LoRAパラメータを設定する  

In [ ]:
from peft import LoraConfig, PeftModel
################################################################################
# LoRA parameters
################################################################################

# LoRA rank dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["query_key_value"]
)

LoRAでFine-Tuningするためには56GBのGPUメモリが必要.  
このメモリが乗る場合はOKだが乗らない場合バッチサイズを減らすか、学習方法を変える必要がある.  
演習環境は24GB,(Google colab無料枠17GB)であるため、LoRAを量子化したQLoRAという学習手法を紹介する.

ここでの量子化(Quantize)とはfloat型(16 bit)で扱っていたパラメタをそれよりの小さい型(8 bitや4 bit)で扱うこと.

---
### QLoRA

QLoRAのポイントは以下の3つ
- 4bit NormalFloat: 64個のパラメタ群を標準化して4bitで表現する,  
→パラメタ64個が4bit, 64個のパラメタ群の量子化定数が32bitでそれぞれ表現される.   
量子化前：32bit * 64 = 2048bit  
量子化後： ４bit * 64 + 32bit = 288bit

- Double Quantization: 4bit NormalFloatで表現された量子化定数群を256個ごとにさらに量子化する
→量子化定数が8bit, 256個のパラメタ群の量子化定数が32bit,でそれぞれ表現される.  
量子化前：４bit * 64 + (32bit) = 288bit  
量子化後：４bit * 64 + (8bit + 32bit * 64/256) = 272bit


- Paged Optimizers
CPUにメモリの一部を乗せておくことでGPU内のメモリ使用量を削減する.

これらの手法によりメモリ使用量を削減することができ学習に必要なメモリを概算すると以下のようになる.  
7b * 4bit / 8  = 3.5GB  

3.5GBのモデルサイズで学習するとバッチサイズ4とすると3.5GB*4 = 14GBのメモリが必要.  
colab無料枠は17GBであるため、学習可能

量子化実装リンク:[bitsandbytes](https://github.com/timdettmers/bitsandbytes)

In [ ]:
from transformers import BitsAndBytesConfig
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

学習パラメタの設定

In [ ]:
# 学習に使うパラメータを設定します

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-5
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.01
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 50
# Log every X updates steps
logging_steps = 50

SFTのパラメタを設定する

In [ ]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False
# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
#学習に使うパラメータを元にTrainerを作成
from trl import SFTTrainer
from transformers import TrainingArguments

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Your GPU supports bfloat16: accelerate training with bf16=True


/home/nagazumi/anaconda3/lib/python3.9/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/nagazumi/anaconda3/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Loading cached processed dataset at /home/nagazumi/.cache/huggingface/datasets/kunishou___json/kunishou--databricks-dolly-15k-ja-6459e484ab1ecf5e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-19aff4628778064f.arrow


実際にトレーニングを行う

In [ ]:
# Train model
trainer.train()
# Fine-tuned model name
new_model = "japanese-stablelm-7b-miniguanaco"
# Save trained model
trainer.model.save_pretrained(new_model)

{'loss': 2.6394, 'learning_rate': 2e-05, 'epoch': 0.2}
{'loss': 2.3556, 'learning_rate': 2e-05, 'epoch': 0.4}
{'loss': 2.2746, 'learning_rate': 2e-05, 'epoch': 0.6}
{'loss': 2.2077, 'learning_rate': 2e-05, 'epoch': 0.8}
{'loss': 2.2361, 'learning_rate': 2e-05, 'epoch': 1.0}
{'train_runtime': 218.8989, 'train_samples_per_second': 4.568, 'train_steps_per_second': 1.142, 'train_loss': 2.342683288574219, 'epoch': 1.0}


テストとして学習

In [ ]:
dataset_test = load_dataset("kunishou/databricks-dolly-15k-ja", split="train")
dataset_test = dataset_test.select(range(1000,1100))
dataset_test

Found cached dataset json (/home/nagazumi/.cache/huggingface/datasets/kunishou___json/kunishou--databricks-dolly-15k-ja-6459e484ab1ecf5e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Dataset({
    features: ['input', 'instruction', 'output', 'index', 'category'],
    num_rows: 100
})

In [ ]:
test_prompt = generate_prompt(dataset_test[1])
print(test_prompt)
# test_promptの[/INST]以降の部分を削除
test_prompt = test_prompt.split("[/INST] ")[0]+"[/INST] "
test_prompt

<|startoftext|> [INST] 夏は、伝統的に暑い、あるいは暖かい天候を連想させます。地中海沿岸の気候では乾燥した天候を意味し、その他の地域では（特にモンスーンのある東アジアでは）雨の天候を意味する。雨季は、サバンナ気候の中で植生が成長する主要な期間です。雨季が偏西風の季節的な変化と関連している場合、モンスーンと呼ばれる。
大西洋北部では、6月1日から11月30日まで、熱帯低気圧の季節があります。大西洋のハリケーン・シーズンの統計的なピークは9月10日である。北東太平洋では、より広い活動期間がありますが、大西洋と同じような時間枠です。北西太平洋では、2月と3月に最小となり、9月上旬にピークを迎える熱帯低気圧が一年中見られます。北インド海盆では、4月から12月に嵐が最も多く、5月と11月にピークがあります。南半球では、熱帯低気圧のシーズンは11月初旬から4月末までで、ピークは2月中旬から3月上旬です。
アメリカやカナダでは、春から夏にかけて雷雨のシーズンがありますが、秋には10月や11月まで続くこともあります。これらの嵐は雹、強風、竜巻を発生させることがあり、通常は午後から夕方にかけて発生します。
北インド海盆で嵐が多いのはいつ頃ですか？ [/INST] 北インド盆地では、4月から12月までが最も多く、5月と11月にピークがあります。


'<|startoftext|> [INST] 夏は、伝統的に暑い、あるいは暖かい天候を連想させます。地中海沿岸の気候では乾燥した天候を意味し、その他の地域では（特にモンスーンのある東アジアでは）雨の天候を意味する。雨季は、サバンナ気候の中で植生が成長する主要な期間です。雨季が偏西風の季節的な変化と関連している場合、モンスーンと呼ばれる。\n大西洋北部では、6月1日から11月30日まで、熱帯低気圧の季節があります。大西洋のハリケーン・シーズンの統計的なピークは9月10日である。北東太平洋では、より広い活動期間がありますが、大西洋と同じような時間枠です。北西太平洋では、2月と3月に最小となり、9月上旬にピークを迎える熱帯低気圧が一年中見られます。北インド海盆では、4月から12月に嵐が最も多く、5月と11月にピークがあります。南半球では、熱帯低気圧のシーズンは11月初旬から4月末までで、ピークは2月中旬から3月上旬です。\nアメリカやカナダでは、春から夏にかけて雷雨のシーズンがありますが、秋には10月や11月まで続くこともあります。これらの嵐は雹、強風、竜巻を発生させることがあり、通常は午後から夕方にかけて発生します。\n北インド海盆で嵐が多いのはいつ頃ですか？ [/INST] '

In [ ]:
print(test_prompt)

<|startoftext|> [INST] 夏は、伝統的に暑い、あるいは暖かい天候を連想させます。地中海沿岸の気候では乾燥した天候を意味し、その他の地域では（特にモンスーンのある東アジアでは）雨の天候を意味する。雨季は、サバンナ気候の中で植生が成長する主要な期間です。雨季が偏西風の季節的な変化と関連している場合、モンスーンと呼ばれる。
大西洋北部では、6月1日から11月30日まで、熱帯低気圧の季節があります。大西洋のハリケーン・シーズンの統計的なピークは9月10日である。北東太平洋では、より広い活動期間がありますが、大西洋と同じような時間枠です。北西太平洋では、2月と3月に最小となり、9月上旬にピークを迎える熱帯低気圧が一年中見られます。北インド海盆では、4月から12月に嵐が最も多く、5月と11月にピークがあります。南半球では、熱帯低気圧のシーズンは11月初旬から4月末までで、ピークは2月中旬から3月上旬です。
アメリカやカナダでは、春から夏にかけて雷雨のシーズンがありますが、秋には10月や11月まで続くこともあります。これらの嵐は雹、強風、竜巻を発生させることがあり、通常は午後から夕方にかけて発生します。
北インド海盆で嵐が多いのはいつ頃ですか？ [/INST] 


In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=50)
result = pipe(test_prompt)

/home/nagazumi/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print(result[0]["generated_text"])

<|startoftext|> [INST] 夏は、伝統的に暑い、あるいは暖かい天候を連想させます。地中海沿岸の気候では乾燥した天候を意味し、その他の地域では（特にモンスーンのある東アジアでは）雨の天候を意味する。雨季は、サバンナ気候の中で植生が成長する主要な期間です。雨季が偏西風の季節的な変化と関連している場合、モンスーンと呼ばれる。
大西洋北部では、6月1日から11月30日まで、熱帯低気圧の季節があります。大西洋のハリケーン・シーズンの統計的なピークは9月10日である。北東太平洋では、より広い活動期間がありますが、大西洋と同じような時間枠です。北西太平洋では、2月と3月に最小となり、9月上旬にピークを迎える熱帯低気圧が一年中見られます。北インド海盆では、4月から12月に嵐が最も多く、5月と11月にピークがあります。南半球では、熱帯低気圧のシーズンは11月初旬から4月末までで、ピークは2月中旬から3月上旬です。
アメリカやカナダでは、春から夏にかけて雷雨のシーズンがありますが、秋には10月や11月まで続くこともあります。これらの嵐は雹、強風、竜巻を発生させることがあり、通常は午後から夕方にかけて発生します。
北インド海盆で嵐が多いのはいつ頃ですか？ [/INST] インド洋の嵐のピークは、11月中旬から12月上旬です。
[/INST] 夏は、伝統的に暑い、あるいは暖かい天候を連想させます。地中海沿岸の気候では


## 5. 評価指標
### 日本語ベンチマークJGLUEによる評価でスコアを上げるためのFine tuning

- JSQuAD  
    文章読解テスト:文脈として情報が与え、質問をする.文脈内に答えが必ず存在する

流れとしては
1. 数字で評価するための関数を定義する
1. 学習前にスコア計算
1. 学習後
1. 学習後のスコア計算

<|startoftext|>[INST] [題名]:梅雨  
[問題]:梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの 広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。  
[質問]:梅雨がみられるのはどの期間？  
 [/INST][答え]: 5月から7月にかけて   

このような形式で学習、検証をしていきます

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import random
import string
import re
from pprint import pprint
from tqdm.notebook import tqdm

import emoji
import neologdn

from datasets import load_dataset
import datasets
datasets.disable_progress_bar()

from collections import Counter
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

スコアを出す関数を定義

In [ ]:
# @title 指標計算などに関する関数を定義

def normalize(s):
    """
    Normalize a sentence by removing punctuations and emoji.

    Parameters
    ----------
    s : str
    Sentence.

    Returns
    ----------
    Normalized sentence.
    """
    def remove_punc(tokens):
        exclude = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
        exclude += string.punctuation
        exclude = [*exclude]
        return "".join([tok for tok in tokens if tok not in exclude])

    def remove_emoji(text):
        text = "".join(["" if emoji.is_emoji(c) else c for c in text])
        emoji_pattern = re.compile(
            "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            "]+",
            flags=re.UNICODE,
        )
        return emoji_pattern.sub(r"", text)

    return remove_punc(neologdn.normalize(remove_emoji(s)))

# Accuracy算出（完全一致）
def accuracy(prediction: str, ground_truth: str):
    """
    Calculate exact-match accuracy.

    Parameters
    ----------
    prediction : str
    Sentence predicted by a model.

    ground_truth : str
    Sentence of ground truth.

    Returns
    ----------
    Boolean. True if prediction is exactly equal to the ground truth after normalization.
    """
    return normalize(prediction) == normalize(ground_truth)

# F1スコア算出
def f1(prediction: str, ground_truth: str):
    """
    Calculate F1 score by character.

    Parameters
    ----------
    prediction : str
    Sentence predicted by a model.

    ground_truth : str
    Sentence of ground truth.

    Returns
    ----------
    F1 score.

    Note
    ----------
    Calculate this score by character because the result will depend on morphogen parser chosen if by word/token.
    ref. https://fintan.jp/page/9126/、https://www.anlp.jp/proceedings/annual_meeting/2023/pdf_dir/H9-4.pdf
    """
    prediction_chars = [char for char in normalize(prediction)]
    ground_truth_chars = [char for char in normalize(ground_truth)]

    common = Counter(prediction_chars) & Counter(ground_truth_chars)
    num_same = sum(common.values())

    if num_same == 0:
        return 0
    precision = num_same / len(prediction_chars)
    recall = num_same / len(ground_truth_chars)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# 予測値と正解のデータセットに対して指標計算を走らせる
def calculate_score(score_func, predictions: list, ground_truths: list):
    """
    Calculate metrics for datasets of predictions and ground_truths.

    Parameters
    ----------
    predictions : list
    List of sentence predicted by a model.

    ground_truths : list
    List of sentence of ground truth.

    Returns
    ----------
    Score(Metrics).
    """
    score = total = 0
    for pred, gtruth in zip(predictions, ground_truths):
        total += 1
        if isinstance(gtruth, list): #JSQuADは質問毎に解答が3つ存在するので、その中で最もスコアが大きいものを採用する(lm-evaluation-harness-jp-stableと同様の手法)
            stored_scores = []
            for gt in gtruth:
                sc = score_func(pred, gt)
                stored_scores.append(sc)
            score += max(stored_scores)
        else:
          score += score_func(pred, gtruth)

    return score / total

JSQuADの形式を入力に変換する関数を定義

In [ ]:

import time
# JSQuAD: 文脈に基づいて質問に回答する（問題文から答えを抜き出す）
"""
次のコードを参考にしている
　- https://github.com/tdc-yamada-ya/lm-evaluation-harness-jp-stable/blob/jp-stable/lm_eval/tasks/jsquad.py
"""

class JSQuAD():
    DESCRIPTION_JSQUAD = "[題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい。\n\n"
    SEED = 42

    def __init__(self):
        ds = load_dataset("shunk031/JGLUE", name="JSQuAD")
        self.ds_train = list(map(self.process_doc, ds['train']))
        self.ds_valid = list(map(self.process_doc, ds['validation']))
        self.metrics = [accuracy, f1] # 完全一致によるAccuracy、および、文字単位でのF1スコアで評価する

    def process_doc(self, doc):
        return {
            "title": doc["title"],
            "context": doc["context"],
            "question": doc["question"],
            "answers": doc["answers"]["text"]
        }

    def doc_to_text(self, doc):
        B_INST, E_INST = "[INST]", "[/INST]"
        B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
        DESCRIPTION_JSQUAD = "[題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい"
        text = f"[題名]:{doc['title']}\n"\
            + f"[問題]:{doc['context'].split('[SEP]')[-1].strip()}\n"\
            + f"[質問]:{doc['question']}\n"

        prompt = "{bos_token}{b_inst} {prompt} {e_inst} ".format(
            bos_token=tokenizer.bos_token,
            b_inst=B_INST,
            system=f"{B_SYS}{DESCRIPTION_JSQUAD}{E_SYS}",
            prompt=text,
            e_inst=E_INST+ "[答え]: "# + doc["answers"]["text"][0],
        )
        return prompt

    def doc_to_answer(self, doc) -> list:
        return doc['answers']

    def fewshot_examples(self, num_fewshot=0, seed=SEED):
        if num_fewshot == 0:
            labeled_examples = ""
        else:
            #trainデータからランダムにnum_fewshotの数だけサンプル抽出
            fewshotex = random.Random(seed).sample(self.ds_train, num_fewshot)

            labeled_examples = (
                "-------------\n"
                + "[例]:\n"
                + "\n\n".join(
                    [
                        self.doc_to_text(doc) + self.doc_to_answer(doc)[0] #doc_to_answer()のoutputはリストであることに注意
                        for doc in fewshotex
                    ]
                )
                + "\n-------------"
                + "\n\n"
            )

        return labeled_examples

    def get_query(self, doc, num_fewshot=0, description=DESCRIPTION_JSQUAD, seed=SEED):
        return (
            description
            + self.fewshot_examples(num_fewshot=num_fewshot, seed=seed)
            + self.doc_to_text(doc)
        )

スコア関数を使い全体の評価を計算する関数を定義

In [ ]:
# @title 評価指標の出力に関する関数を定義

# ランダムサンプリングして小さなデータセットを作成する
def make_small_data(data, n, seed=42):
    """
    Create small dataset.

    Parameters
    ----------
    data : list

    n : int
    Number of data to be extracted by random sampling.

    seed : int
    Random seed.

    Returns
    ----------
    small_data : list
    List of data.
    """
    random.Random(seed).shuffle(data)
    small_data = data[:n]

    return small_data


# 特定のモデル・タスク・shot数（0,1,...）に対する評価指標を算出する
def evaluate(chat_func, num_fewshot, task_name):
    """
    Evaluate LLM on specific task such as JCommonsenseQA and JSQuAD.

    Parameters
    ----------
    chat_func : function

    num_fewshot : int
    Number of examples to be added to query.

    task_name : str
    Task to evaluate LLM such as "JCommonsenseQA" and "JSQuAD".

    Returns
    ----------
    scores : dictionary
    Dictionary of scores for the task.
    """

    N = 8 #モデル評価に用いるvalidationデータ数

    task_dict = {
        #'JCommonsenseQA': JCommonsenseQA,
        'JSQuAD': JSQuAD
    } #対応可能なタスクの一覧

    # タスクを設定
    try:
        task = task_dict[task_name]()
    except KeyError:
        print("Available tasks:")
        pprint([t for t in task_dict.keys()])
        raise KeyError(f"Missing task {task_name}")

    # データセットの読み込み
    ds_valid = make_small_data(task.ds_valid, N) #計算時間短縮のため、精度検証にはvalidationデータからN件抽出して用いる

    # LLMに投げる質問と正解の出力
    queries = list(map(task.get_query, ds_valid, [num_fewshot]*len(ds_valid)))
    answers = list(map(task.doc_to_answer, ds_valid))

    # LLMの回答を出力
    responses = list(map(chat_func, tqdm(queries, leave=False)))

    # 評価指標の算出と出力
    scores = {}
    for score_func in task.metrics:
        score = calculate_score(score_func, responses, answers)
        print(responses, answers)
        scores[score_func.__name__] = score

    return scores


# 指定したzero/few-shot、および、タスクの条件に従い、評価指標の算出結果を出力する
def get_evaluation_results(chat_func,
                           shot_mode: list=['zero', 'few'],
                           task_list: list=['JSQuAD']):
                           #task_list: list=['JCommonsenseQA', 'JSQuAD']):
    """
    Print out the result of evaluation for LLM.

    Parameters
    ----------
    chat_func : function

    shot_mode : list
    List that can only have "zero" or "few" as the element.
    Choose "zero" when you want to evaluate model with zero-shot and "few" when do with few-shot.

    task_list : list
    Task list you may want to evaluate LLM.
    """
    NUM_FEWSHOT = 1

    print("========================================")
    print("[LLMの定量評価結果]")
    print(f"生成関数（モデル）: {chat_func.__name__}")

    for mode in shot_mode:
        assert mode in ['zero', 'few'], 'List of shot_mode should only include "zero" or "few".'
        num_fewshot = 0 if mode == 'zero' else NUM_FEWSHOT

        print(f"{mode}-shot(例の数{num_fewshot}個)による実行結果：")

        for i, task in enumerate(task_list):
            print(f" {i+1}.{task}: {evaluate(chat_func, num_fewshot, task)}")

        print()
    print("========================================")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

modelの回答を出力する関数を定義

In [ ]:
# @title modelの回答を出力する関数

# modelによる回答を出力する
def run_model(question, model=model):
  #inputs = tokenizer(question, return_tensors="pt").input_ids.cuda()
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=50)
  result = pipe(question)
  result = result[0]["generated_text"]
  # 質問文も含めて出力してしまうため、一旦"\n[答え]:"の後を回答として出力する
  response_text = result.split('[答え]:')[-1]

  response_text = response_text.split('\n')[0]
  print("##############")
  print(response_text)
  return response_text

学習前に評価を実行

In [ ]:
get_evaluation_results(run_model, shot_mode=['zero', 'few'])

[LLMの定量評価結果]
生成関数（モデル）: run_model
zero-shot(例の数0個)による実行結果：


Found cached dataset jglue (/home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262)


  0%|          | 0/8 [00:00<?, ?it/s]

/home/nagazumi/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


##############
  [建築]と[土木]のダブルディグリーの機会を提供する新しい教育制度
##############
5月から7月にかけて
##############
  [INST] [題名]:ポルトガル
##############
  [問題]:この改元は改元の決定から日付までを幕府側の事実上の指定で決められたものである。これは事実上の将軍の代始改元を目指したものであるが、以後は幕府権力の衰退もありこれが最後となった。
##############
  [RIAI]
##############
  [題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい。.
##############
  [題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい。.
##############
  [INST] [題名]:ラオス
['  [建築]と[土木]のダブルディグリーの機会を提供する新しい教育制度', '5月から7月にかけて', '  [INST] [題名]:ポルトガル', '  [問題]:この改元は改元の決定から日付までを幕府側の事実上の指定で決められたものである。これは事実上の将軍の代始改元を目指したものであるが、以後は幕府権力の衰退もありこれが最後となった。', '  [RIAI]', '  [題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい。.', '  [題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい。.', '  [INST] [題名]:ラオス'] [['教育', '教育', '教育制度'], ['5月から7月にかけて', '5月から7月にかけて', '5月から7月'], ['リスボン、ポルト、ファロ', 'リスボン、ポルト、ファロ', '空港'], ['将軍', '将軍', '将軍'], ['RIAI', 'RIAI', 'RIAI'], ['フロンテージ', 'フロンテージ', 'フロンテージ'], ['シェーンベルク', 'シェーンベルク', 'シェーンベルク'], ['9世紀頃', '9世紀頃', '9世紀頃']]
['  [建築]と[土木]のダブルディグリーの機会を提供する新しい教育制度', '5月から7月にかけて', '  [INST] [題名]:ポルトガル', 

Found cached dataset jglue (/home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262)


  0%|          | 0/8 [00:00<?, ?it/s]

##############
  建築家
##############
  5月から7月にかけて
##############
  リスボン、ポルト、ファロ
##############
  徳川吉宗
##############
  RIAI
##############
  住居番号
##############
  グスタフ・マーラー
##############
  19世紀後半
['  建築家', '  5月から7月にかけて', '  リスボン、ポルト、ファロ', '  徳川吉宗', '  RIAI', '  住居番号', '  グスタフ・マーラー', '  19世紀後半'] [['教育', '教育', '教育制度'], ['5月から7月にかけて', '5月から7月にかけて', '5月から7月'], ['リスボン、ポルト、ファロ', 'リスボン、ポルト、ファロ', '空港'], ['将軍', '将軍', '将軍'], ['RIAI', 'RIAI', 'RIAI'], ['フロンテージ', 'フロンテージ', 'フロンテージ'], ['シェーンベルク', 'シェーンベルク', 'シェーンベルク'], ['9世紀頃', '9世紀頃', '9世紀頃']]
['  建築家', '  5月から7月にかけて', '  リスボン、ポルト、ファロ', '  徳川吉宗', '  RIAI', '  住居番号', '  グスタフ・マーラー', '  19世紀後半'] [['教育', '教育', '教育制度'], ['5月から7月にかけて', '5月から7月にかけて', '5月から7月'], ['リスボン、ポルト、ファロ', 'リスボン、ポルト、ファロ', '空港'], ['将軍', '将軍', '将軍'], ['RIAI', 'RIAI', 'RIAI'], ['フロンテージ', 'フロンテージ', 'フロンテージ'], ['シェーンベルク', 'シェーンベルク', 'シェーンベルク'], ['9世紀頃', '9世紀頃', '9世紀頃']]
 1.JSQuAD: {'accuracy': 0.375, 'f1': 0.465625}



次に先ほどの評価指標についてスコアを上げることを目的として学習の流れをもう一度確認します.

In [ ]:
#shunk031/JGLUEからJSQuADのデータセットを読み込む    スコア算出ではvalidation(4.44k rows)を使用するのでtrinを使用
dataset = load_dataset("shunk031/JGLUE", "JSQuAD", split="train")

Found cached dataset jglue (/home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262)


In [ ]:
dataset[2]

{'id': 'a1000888p0q2',
 'title': '造語',
 'context': '造語 [SEP] 造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新たな意味の語を造ること、また、そうして造られた語である。新たに造られた語については、新語または新造語とも呼ばれる。',
 'question': 'たに語（単語）を造ることや、既存の語を組み合わせて新たな意味の語を造ること、また、そうして造られた語を意味する言葉は？',
 'answers': {'text': ['造語'], 'answer_start': [0]},
 'is_impossible': False}

In [ ]:
def generate_prompt(data_point):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    DESCRIPTION_JSQUAD = "[題名]と[問題]から[質問]に対する[答え]を名詞で抜き出しなさい"
    text = f"[題名]:{data_point['title']}\n"\
        + f"[問題]:{data_point['context'].split('[SEP]')[-1].strip()}\n"\
        + f"[質問]:{data_point['question']}\n"

    prompt = "{bos_token}{b_inst} {prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DESCRIPTION_JSQUAD}{E_SYS}",
        prompt=text,
        e_inst=E_INST+ "[答え]: " + data_point["answers"]['text'][0],
    )
    return prompt

# テキスト列の追加
def add_text(example):
    example["text"] = generate_prompt(example)
    del example["id"]
    del example["title"]
    del example["context"]
    del example["question"]
    del example["answers"]
    del example["is_impossible"]
    return example
dataset = dataset.map(add_text)

Loading cached processed dataset at /home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262/cache-75d6426c86e6b2d0.arrow


In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 62859
})

In [ ]:
print(dataset[2]['text'])

<|startoftext|>[INST] [題名]:造語
[問題]:造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新たな意味の語を造ること、また、そうして造られた語である。新たに造られた語については、新語または新造語とも呼ばれる。
[質問]:たに語（単語）を造ることや、既存の語を組み合わせて新たな意味の語を造ること、また、そうして造られた語を意味する言葉は？
 [/INST][答え]: 造語 


In [ ]:
#学習時間を短くするために、データセットを1000程度に件に絞る
dataset_small = dataset.select(range(0,1000))

In [ ]:
dataset_small

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_small,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/home/nagazumi/anaconda3/lib/python3.9/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


/home/nagazumi/anaconda3/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Loading cached processed dataset at /home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262/cache-c6ec7a2eb32f670f.arrow


In [ ]:
# Fine-tuned model name
jglue_model = "japanese-stablelm-7b-jglue"

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(jglue_model)

{'loss': 2.7509, 'learning_rate': 2e-05, 'epoch': 0.2}
{'loss': 2.2118, 'learning_rate': 2e-05, 'epoch': 0.4}
{'loss': 1.9653, 'learning_rate': 2e-05, 'epoch': 0.6}
{'loss': 1.935, 'learning_rate': 2e-05, 'epoch': 0.8}
{'loss': 1.8956, 'learning_rate': 2e-05, 'epoch': 1.0}
{'train_runtime': 173.8936, 'train_samples_per_second': 5.751, 'train_steps_per_second': 1.438, 'train_loss': 2.151725006103516, 'epoch': 1.0}


In [ ]:
get_evaluation_results(run_model, shot_mode=['zero', 'few'])

[LLMの定量評価結果]
生成関数（モデル）: run_model
zero-shot(例の数0個)による実行結果：


Found cached dataset jglue (/home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262)


  0%|          | 0/8 [00:00<?, ?it/s]

/home/nagazumi/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


##############
 ダブルディグリーの
##############
5月から7月にかけて
##############
  [INST]:ポルトガル
##############
  [徳川吉宗]
##############
  RIAI
##############
  フロンテージ [/INST][問題]: 街区周辺を市町村の中心に近い角を起点にし、そこから街区の外周に沿って時計回りに距離を測って10 m（ - 15 m）ごとに区切り
##############
  グスタフ・マーラー
##############
  [問題]:このような標高による住み分け分布ができたのは、紀元前からモン・クメール系の人々がこの地域に暮らしていたが、9世紀頃からタイ系の人々が南下してきた。その後、清代末期の19世紀後半からモン
[' ダブルディグリーの', '5月から7月にかけて', '  [INST]:ポルトガル', '  [徳川吉宗]', '  RIAI', '  フロンテージ [/INST][問題]: 街区周辺を市町村の中心に近い角を起点にし、そこから街区の外周に沿って時計回りに距離を測って10\xa0m（ - 15\xa0m）ごとに区切り', '  グスタフ・マーラー', '  [問題]:このような標高による住み分け分布ができたのは、紀元前からモン・クメール系の人々がこの地域に暮らしていたが、9世紀頃からタイ系の人々が南下してきた。その後、清代末期の19世紀後半からモン'] [['教育', '教育', '教育制度'], ['5月から7月にかけて', '5月から7月にかけて', '5月から7月'], ['リスボン、ポルト、ファロ', 'リスボン、ポルト、ファロ', '空港'], ['将軍', '将軍', '将軍'], ['RIAI', 'RIAI', 'RIAI'], ['フロンテージ', 'フロンテージ', 'フロンテージ'], ['シェーンベルク', 'シェーンベルク', 'シェーンベルク'], ['9世紀頃', '9世紀頃', '9世紀頃']]
[' ダブルディグリーの', '5月から7月にかけて', '  [INST]:ポルトガル', '  [徳川吉宗]', '  RIAI', '  フロンテージ [/INST][問題]: 街区周辺を市町村の中心

Found cached dataset jglue (/home/nagazumi/.cache/huggingface/datasets/shunk031___jglue/JSQuAD/1.1.0/1ced89c0df5e89a46cc50032fa7ff73ace35706e86a6d533de4b222334115262)


  0%|          | 0/8 [00:00<?, ?it/s]

##############
  建築家としても土木技師としてもダブルディグリーの機会を提供する新しい教育制度
##############
  5月から7月にかけて来る曇りや雨の多い期間
##############
  リスボン、ポルト、ファロ
##############
  徳川吉宗
##############
  RIAI
##############
  フロンテージ
##############
  グスタフ・マーラー
##############
  19世紀後半
['  建築家としても土木技師としてもダブルディグリーの機会を提供する新しい教育制度', '  5月から7月にかけて来る曇りや雨の多い期間', '  リスボン、ポルト、ファロ', '  徳川吉宗', '  RIAI', '  フロンテージ', '  グスタフ・マーラー', '  19世紀後半'] [['教育', '教育', '教育制度'], ['5月から7月にかけて', '5月から7月にかけて', '5月から7月'], ['リスボン、ポルト、ファロ', 'リスボン、ポルト、ファロ', '空港'], ['将軍', '将軍', '将軍'], ['RIAI', 'RIAI', 'RIAI'], ['フロンテージ', 'フロンテージ', 'フロンテージ'], ['シェーンベルク', 'シェーンベルク', 'シェーンベルク'], ['9世紀頃', '9世紀頃', '9世紀頃']]
['  建築家としても土木技師としてもダブルディグリーの機会を提供する新しい教育制度', '  5月から7月にかけて来る曇りや雨の多い期間', '  リスボン、ポルト、ファロ', '  徳川吉宗', '  RIAI', '  フロンテージ', '  グスタフ・マーラー', '  19世紀後半'] [['教育', '教育', '教育制度'], ['5月から7月にかけて', '5月から7月にかけて', '5月から7月'], ['リスボン、ポルト、ファロ', 'リスボン、ポルト、ファロ', '空港'], ['将軍', '将軍', '将軍'], ['RIAI', 'RIAI', 'RIAI'], ['フロンテージ', 'フロンテージ', 'フロンテージ'], ['シェーンベルク', 'シェーンベルク', 'シェーンベルク'], 

## 参考文献
[[1]](https://arxiv.org/abs/2307.09288)Hugo Touvron, Thomas Scialom, et al. (2023). Llama 2: Open Foundation and Fine-Tuned Chat Models.	arXiv:2307.09288

[[2]](https://arxiv.org/abs/2106.09685)Edward J. Hu, et al. (2021) "LoRA: Low-Rank Adaptation of Large Language Models" Edward J. Hu, et al. arXiv:2106.09685

[[3]](https://arxiv.org/abs/2305.14314)Tim Dettmers, et al. (2023) "QLoRA: Efficient Finetuning of Quantized LLMs" arXiv:2305.14314

[[4]](https://d223302.github.io/AACL2022-Pretrain-Language-Model-Tutorial/lecture_material/AACL_2022_tutorial_PLMs.pdf) Cheng-Han Chiang et al. (2023) "Recent Advances in Pre-trained Language Models:
Why Do They Work and How to Use Them"

[[5]](https://lightning.ai/pages/community/article/lora-llm/)Sebastian Raschka (2023) "Parameter-Efficient LLM Finetuning With Low-Rank Adaptation (LoRA)"

[[6]](https://mlabonne.github.io/blog/posts/Fine_Tune_Your_Own_Llama_2_Model_in_a_Colab_Notebook.html) Maxime Labonne (2023) "Fine-Tune Your Own Llama 2 Model in a Colab Notebook A practical introduction to LLM fine-tuning"

追記
コードのmodel_nameとトークナイザを変更すれば他のモデルでも利用可能です

In [ ]:
#"elyza/ELYZA-japanese-Llama-2-7b"
model_name = "elyza/ELYZA-japanese-Llama-2-7b"
# load tokenizer for ELYZA-japanese-Llama-2-7b
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)